<a href="https://colab.research.google.com/github/edatkinson/Prohibited-Item-Detection/blob/main/x_ray_cnn_binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# Check whether we have a GPU.  Use it if we do.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cpu')

In [ ]:
from google.colab import drive
import pandas as pd
from PIL import Image
import os
from torch.utils.data import Dataset

# Mount Google Drive
drive.mount('/content/drive')

# Navigate to the folder
data_dir = '/content/drive/MyDrive/x_ray_dataset'

# Define transforms
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

# Create dataset class
class X_ray_dataset(Dataset):
    def __init__(self, annotations_file = os.path.join(data_dir, 'binary_classes.csv'), img_dir = os.path.join(data_dir, 'x_ray_images'), transform = None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = Image.open(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Test to see if the class is working
x_ray = X_ray_dataset(transform=transform)
print('len dataset:', len(x_ray)) # use the __len__ method to show the length of the dataset
example = x_ray[0] # use the __getitem__ method to get the first example
print(example[0].shape)
print('Class:', example[1])

len dataset: 9841
torch.Size([3, 256, 256])
Class: 1


In [ ]:
from torch.utils.data import random_split, DataLoader

x_ray = X_ray_dataset(transform=transform)
# Define the size of the training set
train_size = int(0.85 * len(x_ray))  # 85% for training

# Split the dataset into training and testing sets
train_ds, test_ds = random_split(x_ray, [train_size, len(x_ray) - train_size])

# Split the training set into train and validation (train ~ 70% (6887), test ~ 15% (1477), val ~ 15% (1477))
train_ds, val_ds = random_split(train_ds, [6887, 1477])

# Define class labels
classes = ('Safe', 'Prohibited')

batch_size = 10

# Define Dataloaders
train_loader = DataLoader(train_ds,
                          batch_size = batch_size,
                          shuffle = True)
test_loader = DataLoader(test_ds,
                         batch_size = batch_size*5,
                         shuffle = False)
val_loader = DataLoader(val_ds,
                        batch_size = batch_size*5,
                        shuffle = False)

In [ ]:
import torch.nn as nn

# Create CNN
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    # Creating layers
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 128, kernel_size = 3)
    self.batch1 = nn.BatchNorm2d(128)
    self.relu1 = nn.ReLU()
    #self.pool1 = nn.MaxPool2d(2,2)

    self.conv2 = nn.Conv2d(in_channels = 128, out_channels = 10, kernel_size = 3)
    self.batch2 = nn.BatchNorm2d(10)
    self.relu2 = nn.ReLU()
    self.pool2 = nn.AdaptiveAvgPool2d(1)

    #self.conv3 = nn.Conv2d(in_channels = 512, out_channels = 1024, kernel_size = 3)
    #self.pool3 = nn.MaxPool2d(2,2)

    #self.conv4 = nn.Conv2d(in_channels = 1024, out_channels = 2048, kernel_size = 3)
    #self.pool4 = nn.MaxPool2d(2,2)

    #self.conv5 = nn.Conv2d(in_channels = 2048, out_channels = 4096, kernel_size = 3)
    #self.pool5 = nn.MaxPool2d(2,2)

    #self.conv6 = nn.Conv2d(in_channels = 4096, out_channels = 8192, kernel_size = 3)
    #self.pool6 = nn.MaxPool2d(2,2)

    # flatten (make it a vector) so can make normal linear layers like a fully connected NN
    #self.flatten = nn.Flatten()

    #self.fc1 = nn.Linear(in_features = 1968128, out_features = 2048)
    #self.drop1 = nn.Dropout(p = 0.3)

    #self.fc2 = nn.Linear(in_features = 2048, out_features = 1024)
    #self.drop2 = nn.Dropout(p = 0.3)

    # Output Layer ending in 2 values (rough probability distribution over the 2 classes)
    #self.out = nn.Linear(in_features = 512, out_features = 2)


  def forward(self, x):
    x = nn.functional.relu(self.conv1(x)) # apply non-linearity
    x = self.batch1(x)
    x = self.relu1(x)
    #x = self.pool1(x)

    x = nn.functional.relu(self.conv2(x)) # apply non-linearity
    x = self.batch2(x)
    x = self.relu2(x)
    x = self.pool2(x)

    #x = nn.functional.relu(self.conv3(x)) # apply non-linearity
    #x = self.pool3(x)

    #x = nn.functional.relu(self.conv4(x)) # apply non-linearity
    #x = self.pool4(x)

    #x = nn.functional.relu(self.conv5(x)) # apply non-linearity
    #x = self.pool5(x)

    #x = nn.functional.relu(self.conv6(x)) # apply non-linearity
    #x = self.pool6(x)

    #x = self.flatten(x)

    #x = nn.functional.relu(self.fc1(x))
    #x = self.drop1(x) # dropout to prevent overfitting

    #x = nn.functional.relu(self.fc2(x))
    #x = self.drop2(x) # dropout to prevent overfitting

    #x = self.out(x)

    return x

In [ ]:
# Instantiate Network
net = CNN()
net.to(device)

CNN(
  (conv1): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1))
  (batch1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(128, 10, kernel_size=(3, 3), stride=(1, 1))
  (batch2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): AdaptiveAvgPool2d(output_size=1)
  (out): Linear(in_features=512, out_features=2, bias=True)
)

In [ ]:
# Checking the shape of an image before and after the network
for i, data in enumerate(train_loader):
  inputs, labels = data[0].to(device), data[1].to(device)
  print('Input shape:', inputs.shape)
  print('Output shape:', net(inputs).shape)
  break

Input shape: torch.Size([10, 3, 128, 128])
Output shape: torch.Size([10, 10, 1, 1])


In [ ]:
# Count the number of learnable parameters
num_pars = 0
for i in net.parameters():
  num_pars += len(torch.flatten(i))

print(f'Number of parameters in the model: {num_pars:,}')

Number of parameters in the model: 31,776


In [ ]:
# Specifying hyperparameters
import torch.optim as optim

loss_func = nn.CrossEntropyLoss()
optimiser = optim.SGD(net.parameters(), lr = 0.1)

In [ ]:
# Function for training one epoch
def train_epoch():
  net.train(True) # set to training mode

  # Metrics that will build up
  running_loss = 0
  running_accuracy = 0

  # Iterate over train data
  for batch_index, data in enumerate(train_loader):
    inputs, labels = data[0].to(device), data[1].to(device) # get the images and labels

    optimiser.zero_grad() # set all non-zero values for gradients to 0 (reset gradients)

    outputs = net(inputs).squeeze((-1, -2)) # shape: [batch size, 2]
    correct_prediction = torch.sum(labels == torch.argmax(outputs, dim = 1)).item() # check how many images the model predicted correctly
    running_accuracy += correct_prediction/batch_size # update the accuracy

    # Train model
    loss = loss_func(outputs, labels) # compare model predictions with labels
    running_loss += loss.item() # update the loss
    loss.backward() # calculate gradients
    optimiser.step()

    if batch_index % 50 == 49:      # print for every 500 batchs
      avg_loss = running_loss/50  # get the average loss across batches
      avg_acc = (running_accuracy/50) * 100 # get the average accuracy across batches
      print('Batch {0}, Loss: {1:.3f}, Accuracy: {2:.1f}%'.format(batch_index+1, avg_loss, avg_acc))

      running_loss = 0
      running_accuracy = 0

  print()


In [ ]:
# Function for validating one epoch
def validate_epoch():
  net.train(False) # set to evaluation mode
  running_loss = 0
  running_acc = 0

  # Iterate over validation data
  for batch_index, data in enumerate(val_loader):
    inputs, labels = data[0].to(device), data[1].to(device)

    with torch.no_grad(): # not worried about gradients here as not training
      outputs = net(inputs).squeeze((-1, -2)) # shape [batch size, 2]
      correct_prediction = torch.sum(labels == torch.argmax(outputs, dim = 1)).item() # check how many images the model predicted correctly
      running_acc += correct_prediction/batch_size # update the accuracy
      loss = loss_func(outputs, labels) # compare model predictions with labels
      running_loss += loss.item() # update the loss

  avg_loss = running_loss/len(val_loader)
  avg_acc = (running_acc/len(val_loader)) * 100

  print('Val Loss: {0:.3f}, Val Accuracy: {1:.1f}%'.format(avg_loss, avg_acc))

  print('-----------------------------------------------------------')
  print()



In [ ]:
# Function for testing
def test_epoch(epoch):
  with torch.no_grad():
    correct_prediction = 0
    total = 0
    for images, labels in test_loader:
      images = images.to(device)
      labels = labels.to(device)

      outputs = net(images).squeeze((-1,-2))

      predicted = torch.argmax(outputs, -1)
      correct_prediction += (predicted == labels).sum().item()
      total += labels.size(0)

    acc =(correct_prediction/total) * 100

    print('Test accuracy after {0:.0f} epoch(s): {1:.1f}%'.format(epoch+1, acc))
    print()

In [ ]:
# Training loop

num_epochs = 5

for i in range(num_epochs):
  print('Epoch:', i+1, '\n')

  train_epoch()
  test_epoch(i)
  validate_epoch()

print('Finished Training')

Epoch: 1 

Batch 50, Loss: 0.119, Accuracy: 97.2%
Batch 100, Loss: 0.119, Accuracy: 96.8%
Batch 150, Loss: 0.052, Accuracy: 99.2%
Batch 200, Loss: 0.074, Accuracy: 98.0%
Batch 250, Loss: 0.105, Accuracy: 97.0%
Batch 300, Loss: 0.118, Accuracy: 97.0%
Batch 350, Loss: 0.114, Accuracy: 96.6%
Batch 400, Loss: 0.083, Accuracy: 98.4%
Batch 450, Loss: 0.142, Accuracy: 95.6%
Batch 500, Loss: 0.165, Accuracy: 95.0%
Batch 550, Loss: 0.135, Accuracy: 96.0%
Batch 600, Loss: 0.130, Accuracy: 96.0%
Batch 650, Loss: 0.101, Accuracy: 97.2%

Val Loss: 0.292, Val Accuracy: 465.7%
-----------------------------------------------------------

Test accuracy after 1 epoch(s): 94.38050101557211 %
Epoch: 2 

Batch 50, Loss: 0.074, Accuracy: 98.8%
Batch 100, Loss: 0.136, Accuracy: 96.4%
Batch 150, Loss: 0.128, Accuracy: 95.6%
Batch 200, Loss: 0.122, Accuracy: 97.0%
Batch 250, Loss: 0.130, Accuracy: 96.6%
Batch 300, Loss: 0.104, Accuracy: 96.8%
Batch 350, Loss: 0.055, Accuracy: 98.8%
Batch 400, Loss: 0.130, Accu